# 증권사 API 연결과 매매하기

## 접근 토큰 및 해시키 발급하기

### 접근 토큰 발급받기

In [4]:
import requests
import json
import keyring

# key
app_key = keyring.get_password('koreainvest_mock_app_key', 'ahn283')
app_secret = keyring.get_password('koreainvest_mock_app_secret', 'ahn283')

# base url
url_base = 'https://openapivts.koreainvestment.com:29443'        # 모의투자

# information
headers = {"content-type": "application/json"}
path = "oauth2/tokenP"
body = {
    "grant_type": "client_credentials",
    "appkey": app_key,
    "appsecret": app_secret
}
url = f"{url_base}/{path}"
print(url)



https://openapivts.koreainvestment.com:29443/oauth2/tokenP


In [5]:
# 위 url을 body에 전송하면 접근 토근을 받을 수 있다.
# dumps() 함수를 통해 딕셔너리를 json 문자열로 변환하여 전송
res = requests.post(url, headers=headers, data=json.dumps(body))
access_token = res.json()['access_token']

### 해시키 발급받기

In [6]:
def hashkey(datas):
    path = 'uapi/hashkey'
    url = f'{url_base}/{path}'
    headers = {
        'content-type': 'application/json',
        'appKey': app_key,
        'appSecret': app_secret
    }
    res = requests.post(url, headers=headers, data=json.dumps(datas))
    hashkey = res.json()['HASH']
    
    return hashkey

## 주식 현재가 시세 조회하기

In [7]:
# 삼성전자

path = 'uapi/domestic-stock/v1/quotations/inquire-price'
url = f'{url_base}/{path}'

headers = {
    'Content-type' : 'application/json',
    'authorization' : f'Bearer {access_token}',
    'appKey' : app_key,
    'appSecret' : app_secret,
    # tr_ud는 거래ID에 해당하는 부분으로써 주식현재가 시세에 해당하는 FHKST0101010을 입력한다.
    'tr_id' : 'FHKST01010100'
}
# fid_cond_mrkt_div_code는 시장 분류코드며, 주식에 해당하는 J를 입력하고 fid_input_iscd에는 삼성전자 티커를 입력
params = {'fid_cond_mrkt_div_code': 'J', 'fid_input_iscd': '005930'}

res = requests.get(url, headers=headers, params=params)
res.json()['output']['stck_prpr']

'72000'

## 주식 주문하기

### 매수주문

In [10]:
path = '/uapi/domestic-stock/v1/trading/order-cash'
url = f'{url_base}/{path}'

data = {
    'CANO': '50099655',     # 계좌번호 앞 8자리
    'ACNT_PRDT_CD': '01',   # 계좌번호 뒤 2자리
    'PDNO': '005930',       # 종목코드
    'ORD_DVSN': "01",       # 주문 방법 : 01-시장가 주문
    'ORD_QTY': "10",        # 주문   
    'ORD_UNPR': "0"         # 주문 단가 (시장가의 경우 0)    
}

headers = {
    "Content-Type": "application/json",
    "authorization": f"Bearer {access_token}",
    "appKey": app_key,
    "appSecret": app_secret,
    "tr_id": "VTTC0802U",               # 주식 현금 매수 주문 tr_id
    "custtype": "P",
    "hashkey": hashkey(data)            # 주문 보안을 위해 해시키를 발급받아 입력
}

res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40100000', 'msg1': '모의투자 영업일이 아닙니다.'}